In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")
# 查看数据行列
train.shape

(891, 12)

In [4]:
# 检查NA值
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# 筛选有用特征
train = train.drop(labels=["PassengerId","Name","Cabin","Ticket"],axis=1)
test = test.drop(labels=["Name","Cabin","Ticket"],axis=1)

In [6]:
# 去除有缺失值的行
train=train.dropna()

In [7]:
# 分类变量编码（字符串型转数字型）
train_dummy=pd.get_dummies(train[["Sex","Embarked"]])# 独热编码，和sklearn OneHot功能一样
train_conti = pd.DataFrame(train,columns=["Survived","Fare","Parch","SibSp","Age","Pclass"],index = train.index)
train = train_conti.join(train_dummy)# 数据拼接
test_dummy=pd.get_dummies(test[["Sex","Embarked"]])# 独热编码，和sklearn OneHot功能一样
test_conti = pd.DataFrame(test,columns=["PassengerId","Fare","Parch","SibSp","Age","Pclass"],index = test.index)
test = test_conti.join(test_dummy)# 数据拼接
test.info()
# 将测试集空数据补全
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))
test["Fare"] = test["Fare"].fillna(np.mean(test["Fare"]))
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Fare           417 non-null float64
Parch          418 non-null int64
SibSp          418 non-null int64
Age            332 non-null float64
Pclass         418 non-null int64
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
dtypes: float64(2), int64(4), uint8(5)
memory usage: 21.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Fare           418 non-null float64
Parch          418 non-null int64
SibSp          418 non-null int64
Age            418 non-null float64
Pclass         418 non-null int64
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarke

In [8]:
x = train.iloc[:,1:]# 特征列
y = train.iloc[:,0]# 标签列
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)#数据集划分

In [9]:
# 标准化以保证数据的快速收敛
stdsc = StandardScaler()
x_train_conti_std=stdsc.fit_transform(x_train[["Age","SibSp","Parch","Fare"]])#拟合
x_test_conti_std=stdsc.fit_transform(x_test[["Age","SibSp","Parch","Fare"]])
#将ndarry转化成dataframe
x_train_conti_std=pd.DataFrame(x_train_conti_std,columns=["Age","SibSp","Parch","Fare"],index=x_train.index)
x_test_conti_std=pd.DataFrame(x_test_conti_std,columns=["Age","SibSp","Parch","Fare"],index=x_test.index)

In [10]:
# 使用逻辑回归建模
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)# 模型训练

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# 将模型应用于测试集并查看混淆矩阵
y_pred = classifier.predict(x_test)
# 打印混淆矩阵
confusion_matrix = confusion_matrix(y_test,y_pred)
print(confusion_matrix )

[[105  20]
 [ 28  61]]


In [12]:
# 打印准确率
print("Accuracy :{:.2f}".format(classifier.score(x_test,y_test)))

Accuracy :0.78


In [13]:
print(test)
predict = classifier.predict(test.iloc[:,1:])
submission = pd.DataFrame({'PassengerId':test["PassengerId"],'Survived':predict})# 以字典的形式建立dataframe
submission.to_csv("submmision_csv",index = False)# 转换为CSV文件
pd.read_csv("submmision_csv")

     PassengerId      Fare  Parch  SibSp       Age  Pclass  Sex_female  \
0            892    7.8292      0      0  34.50000       3           0   
1            893    7.0000      0      1  47.00000       3           1   
2            894    9.6875      0      0  62.00000       2           0   
3            895    8.6625      0      0  27.00000       3           0   
4            896   12.2875      1      1  22.00000       3           1   
..           ...       ...    ...    ...       ...     ...         ...   
413         1305    8.0500      0      0  30.27259       3           0   
414         1306  108.9000      0      0  39.00000       1           1   
415         1307    7.2500      0      0  38.50000       3           0   
416         1308    8.0500      0      0  30.27259       3           0   
417         1309   22.3583      1      1  30.27259       3           0   

     Sex_male  Embarked_C  Embarked_Q  Embarked_S  
0           1           0           1           0  
1      

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
